In [2]:
import numpy as np

notebooks/examples/01_lie_se3.ipynb

# Q1 - Coordinate Transforms

In [3]:
# Set as a 3x3 rotation matrix R_{CW} that rotates 30 degrees around the y-axis
rotation_CW = np.array([
    [np.cos(np.pi/6), 0, np.sin(np.pi/6)],
    [0, 1, 0],
    [-np.sin(np.pi/6), 0, np.cos(np.pi/6)]]
    )
# Define the translation vector between coordinate frames t_{CW} = [0.5, 0.0, 1.0]^T
translate_CW = np.array([0.5, 0.0, 1.0])
print("Translate from world to camera: ", translate_CW)
# Pick a world point p_W = [1.0, 0.5, 2.0]^T
point_W = np.array([1.0, 0.5, 2.0])
print("Point in world frame: ", point_W)
# What is the coordinates of p_W in the camera frame?
point_C = rotation_CW @ (point_W - translate_CW)
print("Point in camera frame: ", point_C)
# Verify this through an inverse operation
# Invert the rotation matrix
rotation_WC = rotation_CW.T
# Project onto world frame
point_W_est = (rotation_WC @ point_C) + translate_CW
print("Estimate of point back in world frame: ", point_W_est)

Translate from world to camera:  [0.5 0.  1. ]
Point in world frame:  [1.  0.5 2. ]
Point in camera frame:  [0.9330127 0.5       0.6160254]
Estimate of point back in world frame:  [1.  0.5 2. ]


# Q2 - Homogeneous transform

In [4]:
# Construct the homogeneous transform T_{CW} in SE(E) as a 4x4 matrix
translate_WC = -(rotation_CW @ translate_CW)
transform_CW = np.block([
  [rotation_CW, translate_WC.reshape(3, 1)],
  [np.zeros((1, 3)), np.ones((1,1))]]
)
print("Homogeneous transformation matrix: ", transform_CW)
# Apply it to the same point using homogeneous coordinates
point_W_h = np.hstack((point_W, 1))
point_C_h = transform_CW @ point_W_h
# Compare this with the earlier direct computation
print("Point in camera frame: ", point_C)
print("Point in camera frame (homogeneous coordinates): ", point_C_h)
# Compute T_{WC} as the inverse of T_{CW}
transform_WC = np.block([
  [rotation_WC, translate_CW.reshape(3, 1)],
  [np.zeros((1, 3)), np.ones((1,1))]
])
# Test it recovers p_W
point_W_h = transform_WC @ point_C_h
print("Point in world frame: ", point_W)
print("Point in world frame (homogeneous coordinates): ", point_W_h)

Homogeneous transformation matrix:  [[ 0.8660254  0.         0.5       -0.9330127]
 [ 0.         1.         0.        -0.       ]
 [-0.5        0.         0.8660254 -0.6160254]
 [ 0.         0.         0.         1.       ]]
Point in camera frame:  [0.9330127 0.5       0.6160254]
Point in camera frame (homogeneous coordinates):  [0.9330127 0.5       0.6160254 1.       ]
Point in world frame:  [1.  0.5 2. ]
Point in world frame (homogeneous coordinates):  [1.  0.5 2.  1. ]


# Q3 - Axis-angle and Quaternion

In [6]:
# Convert the rotation into axis, angle form
rotation_axis = np.hstack([0, 1, 0])
rotation_angle = np.pi/6
rotation_axis_angle = np.hstack([rotation_axis, rotation_angle])
print("Rotation axis-angle: ", rotation_axis_angle)
# Convert the rotation matrix into a quaternion
t = np.trace(rotation_CW)
print("Value of t (must be positive): ", np.round(t, 3))
q_w = 0.5*np.sqrt(1 + t)
q_x = (1/(4*q_w))*(rotation_CW[2][1] - rotation_CW[1][2])
q_y = (1/(4*q_w))*(rotation_CW[0][2] - rotation_CW[2][0])
q_z = (1/(4*q_w))*(rotation_CW[1][0] - rotation_CW[2][1])
q = np.hstack([q_w, q_x, q_y, q_z])
q_norm = np.linalg.norm(q)
q = q / q_norm
print("Quaternion rotation: ", np.round(q, 3))
# Check by converting the  quaternion to axis-angle
theta = 2*np.arccos(q_w)
u_x = q_x / np.sin(theta/2)
u_y = q_y / np.sin(theta/2)
u_z = q_z / np.sin(theta/2)
u = np.hstack([u_x, u_y, u_z])
print("Axis: ", np.round(u, 3))
print("Angle (radians): ", np.round(theta, 3))
# Quaternion into rotation matrix
q_w = q[0]
q_x = q[1]
q_y = q[2]
q_z = q[3]
R_q = np.zeros((3, 3), dtype=float)
R_q[0][0] = 1 - 2*(q_y**2 + q_z**2)
R_q[0][1] = 2*(q_x*q_y - q_z*q_w)
R_q[0][2] = 2*(q_x*q_z + q_y*q_w)
R_q[1][0] = 2*(q_x*q_y + q_z*q_w)
R_q[1][1] = 1 - 2*(q_x**2 + q_z**2)
R_q[1][2] = 2*(q_y*q_z - q_x*q_w)
R_q[2][0] = 2*(q_x*q_z - q_y*q_w)
R_q[2][1] = 2*(q_y*q_z + q_x*q_w)
R_q[2][2] = 1 - 2*(q_x**2 + q_y**2)
print("Rotation(q) matrix: ", np.round(R_q, 3))
print("Matches R_CW? ", np.allclose(rotation_CW, R_q))

Rotation axis-angle:  [0.         1.         0.         0.52359878]
Value of t (must be positive):  2.732
Quaternion rotation:  [0.966 0.    0.259 0.   ]
Axis:  [0. 1. 0.]
Angle (radians):  0.524
Rotation(q) matrix:  [[ 0.866  0.     0.5  ]
 [ 0.     1.     0.   ]
 [-0.5    0.     0.866]]
Matches R_CW?  True


# Q4 - SO(3) Lie Algebra and Hat operator

In [7]:
# Define a rotation vector of ~17 degrees around the y-axis
phi = np.hstack([0.0, 0.3, 0.0])
# Implement the hat operator to this
phi_hat = np.zeros((3, 3))
phi_hat[0][1] = - phi[2]
phi_hat[0][2] = phi[1]
phi_hat[1][2] = - phi[0]
phi_hat[1][0] = - phi_hat[0][1]
phi_hat[2][0] = - phi_hat[0][2]
phi_hat[2][1] = - phi_hat[1][2]
print("Phi^: ", phi_hat)
# Verify that the hat operator performs vector cross multiplication
v = np.hstack([2.0, -1.0, 3.0])
phi_hat_v = phi_hat @ v
phi_cross_v = np.cross(phi, v)
print("Hat operator works? ", np.allclose(phi_hat_v, phi_cross_v))
# What is the exponential of phi^
# Well, exp(phi^) = R(phi). We can approximate this with Rodrigues' formula
theta_phi = np.linalg.norm(phi)
R_phi = np.eye(3) + (np.sin(theta_phi)/theta_phi)*phi_hat + ((1-np.cos(theta_phi))/(theta_phi**2))*(phi_hat @ phi_hat)
print("R(phi): ", np.round(R_phi,3))


Phi^:  [[ 0.  -0.   0.3]
 [ 0.   0.  -0. ]
 [-0.3  0.   0. ]]
Hat operator works?  True
R(phi):  [[ 0.955  0.     0.296]
 [ 0.     1.     0.   ]
 [-0.296  0.     0.955]]


# Q5 - SE(3) Lie Algebra and Hat operator

In [8]:
# Construct a 3D twist
rho = np.hstack([0.1, 0.0, 0.2])
xi = np.hstack([rho, phi])
# Create the Jacobian of SO(3)
j_phi = np.eye(3) + (1 - np.cos(theta_phi)/(theta_phi**2))*phi_hat + ((theta_phi-np.sin(theta_phi))/(theta_phi**3))*(phi_hat @ phi_hat)
# Exponentiate phi^ (approximate Rodrigues)
T_xi_approx = np.block([
  [(np.eye(3) + phi_hat), (rho + 0.5*phi_hat @ rho).reshape((3, 1))],
  [np.zeros((1, 3)), np.ones(1)]
])
print("T(xi) approximation: ", np.round(T_xi_approx, 3))
# Exponentiate phi^ (exact Rodrigues)
T_xi_exact = np.block([
[R_phi, (j_phi @ rho).reshape((3, 1))],
[np.zeros((1, 3)), np.ones(1)]
])
print("T(xi) exact: ", np.round(T_xi_exact, 3))
print("T(xi) delta: ", np.round((T_xi_approx - T_xi_exact), 3))

T(xi) approximation:  [[ 1.     0.     0.3    0.13 ]
 [ 0.     1.     0.     0.   ]
 [-0.3    0.     1.     0.185]
 [ 0.     0.     0.     1.   ]]
T(xi) exact:  [[ 0.955  0.     0.296 -0.478]
 [ 0.     1.     0.     0.   ]
 [-0.296  0.     0.955  0.485]
 [ 0.     0.     0.     1.   ]]
T(xi) delta:  [[ 0.045  0.     0.004  0.608]
 [ 0.     0.     0.     0.   ]
 [-0.004  0.     0.045 -0.3  ]
 [ 0.     0.     0.     0.   ]]


# Q6 - Pose Updates via Exponentiation

In [ ]:
# ...